# Configurations for Regression Tasks

In [1]:
import time,nbloader
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import matplotlib.pyplot as plt
from util import grid_maker,get_properIdx,data4imitation_expert_crazy,gpusession
from define_choiceNet_imitation import choiceNet_imitation_class
from define_mdn_imitation import mdn_imitation_class
from define_mlp_imitation import mlp_imitation_class
from multiprocessing import Process,Manager
if __name__ == "__main__": 
    print ("TensorFlow version is [%s]."%(tf.__version__))

TensorFlow version is [1.10.0].


### Configurations

In [2]:
trainModeList = ['MDN'] # ['CN','MLP','MDN']
dataEnvList = ['HalfCheetah-v2']
oRateList = [0.0,0.1,0.2,0.3,0.4] # [0.0,0.2,0.4,0.6]
varList = [1e-6]

### Helper functinos

In [3]:
def get_reg_config(_processID=0,_maxProcessID=8,_maxGPU=8):
    # Get total configurations
    _G = grid_maker(trainModeList,dataEnvList,oRateList,varList)
    _ids = get_properIdx(_processID,_maxProcessID,_nTask=_G.nIter)
    _paramsList = list(_G.paramList[i] for i in _ids) 
    # Get corresponding GPU ID 
    _GPU_ID = (_processID % _maxGPU)
    return _paramsList,_GPU_ID
def get_common_config():
    _n = 500 # Number of training data
    _oRange=[-5.,+5.0]
    _hdims = [32,32] # [64,64] for CN, MLP
    _actv = tf.nn.tanh
    _maxEpoch = 3e4
    _PLOT_EVERY = 0
    _SAVE_NET = False 
    _SAVE_FIG = True 
    return _n,_oRange,_hdims,_actv,_maxEpoch,_PLOT_EVERY,_SAVE_NET,_SAVE_FIG
def run_cn(_trainMode,_EnvType,_oRate,_var,_GPU_ID):
    _n,_oRange,_hdims,_actv,_maxEpoch,_PLOT_EVERY,_SAVE_NET,_SAVE_FIG = get_common_config()
    oRate = [1-_oRate,_oRate]
    x,y,t=data4imitation_expert_crazy(envname=_EnvType,_n=_n,_oRate=oRate,measVar=_var)
    xtest = np.linspace(start=-3,stop=3,num=1000).reshape((-1,1))
    tf.reset_default_graph()
    tf.set_random_seed(0); np.random.seed(0)
    C = choiceNet_imitation_class(_name='CN_%s_E%02d_C%02d_var%.1e'%(_EnvType,oRate[0]*100,oRate[1]*100,_var),_envname=_EnvType
                            ,_xdim=x.shape[1],_ydim=y.shape[1],_hdims=_hdims
                            ,_kmix=5,_actv=_actv,_bn=slim.batch_norm
                            ,_rho_ref_train=0.99,_tau_inv=1e-2,_var_eps=1e-8
                            ,_pi1_bias=0.0,_logSigmaZval=0
                            ,_kl_reg_coef=1e-6,_l2_reg_coef=1e-6
                            ,_SCHEDULE_MDN_REG=False
                            ,_GPU_ID=_GPU_ID,_VERBOSE=False)
    sess = gpusession() # Open session
    sess.run(tf.global_variables_initializer()) # Initialize variables
    C.train(_sess=sess,_x=x,_y=y,_yref=t
           ,_lr=1e-1,_batchSize=256,_maxEpoch=_maxEpoch,_kp=1.0
           ,_LR_SCHEDULE=True
           ,_PRINT_EVERY=50,_PLOT_EVERY=_PLOT_EVERY
           ,_SAVE_TXT=True,_SAVE_BEST_NET=_SAVE_NET,_SAVE_FINAL=_SAVE_NET)
    C.test(_sess=sess,_PLOT_TRAIN=True,_PLOT_RES=True,_SAVE_FIG=_SAVE_FIG)
    sess.close() # Close session
def run_mdn(_trainMode,_EnvType,_oRate,_var,_GPU_ID):
    _n,_oRange,_hdims,_actv,_maxEpoch,_PLOT_EVERY,_SAVE_NET,_SAVE_FIG = get_common_config()
    oRate = [1-_oRate,_oRate]
    x,y,t=data4imitation_expert_crazy(envname=_EnvType,_n=_n,_oRate=oRate,measVar=_var)
    xtest = np.linspace(start=-3,stop=3,num=500).reshape((-1,1))
    tf.reset_default_graph()
    tf.set_random_seed(0); np.random.seed(0)
    MDN = mdn_imitation_class(_name='MDN_%s_E%02d_C%02d_var%.1e'%(_EnvType,oRate[0]*100,oRate[1]*100,_var),_envname=_EnvType
                     ,_xdim=x.shape[1],_ydim=y.shape[1],_hdims=_hdims,_sigmax=0.1
                     ,_kmix=3,_actv=_actv,_bn=slim.batch_norm
                     ,_l2_reg_coef=1e-5,_GPU_ID=_GPU_ID,_VERBOSE=False)
    sess = gpusession() # Open session
    sess.run(tf.global_variables_initializer()) # Initialize variables
    MDN.train(_sess=sess,_x=x,_y=y,_yref=t 
           ,_lr=1e-2,_batchSize=128,_maxEpoch=_maxEpoch,_kp=1.0
           ,_LR_SCHEDULE=True
           ,_PRINT_EVERY=50,_PLOT_EVERY=_PLOT_EVERY
           ,_SAVE_TXT=True,_SAVE_BEST_NET=_SAVE_NET,_SAVE_FINAL=_SAVE_NET)
    MDN.test(_sess=sess,_PLOT_TRAIN=True,_PLOT_RES=True,_SAVE_FIG=_SAVE_FIG)
    sess.close() # Close session
def run_mlp(_trainMode,_EnvType,_oRate,_var,_GPU_ID):
    _n,_oRange,_hdims,_actv,_maxEpoch,_PLOT_EVERY,_SAVE_NET,_SAVE_FIG = get_common_config()
    oRate = [1-_oRate,_oRate]
    x,y,t=data4imitation_expert_crazy(envname=_EnvType,_n=_n,_oRate=oRate,measVar=_var)
    xtest = np.linspace(start=-3,stop=3,num=500).reshape((-1,1))
    tf.reset_default_graph()
    tf.set_random_seed(0); np.random.seed(0)
    MLP =  mlp_imitation_class(_name='MLP_%s_E%02d_C%02d_var%.1e'%(_EnvType,oRate[0]*100,oRate[1]*100,_var),_envname=_EnvType
                         ,_xdim=x.shape[1],_ydim=y.shape[1],_hdims=_hdims,_actv=_actv,_bn=None # slim.batch_norm
                         ,_l2_reg_coef=1e-5,_GPU_ID=_GPU_ID,_VERBOSE=False)
    sess = gpusession() # Open session
    sess.run(tf.global_variables_initializer()) # Initialize variables
    MLP.train(_sess=sess,_x=x,_y=y,_yref=t 
               ,_lr=1e-1,_batchSize=256,_maxEpoch=_maxEpoch,_kp=1.0
               ,_LR_SCHEDULE=True
               ,_PRINT_EVERY=50,_PLOT_EVERY=_PLOT_EVERY
               ,_SAVE_TXT=True,_SAVE_BEST_NET=_SAVE_NET,_SAVE_FINAL=_SAVE_NET)
    MLP.test(_sess=sess,_PLOT_TRAIN=True,_PLOT_RES=True,_SAVE_FIG=_SAVE_FIG)
    sess.close() # Close session
def train_wrapper_reg(_paramsList,_GPU_ID):
    for pIdx,params in enumerate(_paramsList): # For all current configurations
        print ("===[%d/%d]==="%(pIdx,len(_paramsList)))
        _trainMode,_dataType,_oRate,_var = params[0],params[1],params[2],params[3]
        if _trainMode == 'CN':
            run_cn(_trainMode,_dataType,_oRate,_var,_GPU_ID)
        elif _trainMode == 'MDN':
            run_mdn(_trainMode,_dataType,_oRate,_var,_GPU_ID)
        elif _trainMode == 'MLP':
            run_mlp(_trainMode,_dataType,_oRate,_var,_GPU_ID)
        else:
            print ("Unknown trainMode:[%s]."%(_trainMode))
if __name__ == "__main__":
    print ("helper functions defined.")

helper functions defined.


### Worker Class

In [4]:
class worker_class(Process):
    def __init__(self,_idx=0,_maxProcessID=8,_maxGPU=8,_name='worker',_VERBOSE=True):
        Process.__init__(self)
        # Initialize class
        self.idx = _idx
        self.maxProcessID = _maxProcessID
        self.maxGPU = _maxGPU
        self.name = _name
        self.VERBOSE = _VERBOSE
        # Initialize Process
        self.setName = 'T_'+self.name 
        self.daemon = True # Make sure that each child is killed when the parent is dead. (?!)
        # Print
        if self.VERBOSE:
            print ("[%s] Instantiated."%(self.name))
    def run(self):
        print ("Starting [%s]"%(self.name))
        time.sleep(1e-1)
        # Get configurations 
        paramsList,GPU_ID = get_reg_config(self.idx,self.maxProcessID,self.maxGPU)
        print ("processID:[%d/%d] GPU_ID:[%d] #Config:[%d]"
               %(self.idx,self.maxProcessID,GPU_ID,len(paramsList)))
        # Run
        train_wrapper_reg(_paramsList=paramsList,_GPU_ID=GPU_ID)
        print ("[%s] Finished."%(self.name))
if __name__ == "__main__":
    print ("worker_class defined.")

worker_class defined.
